In [103]:
from bs4 import BeautifulSoup as bs
import requests
from tqdm import tqdm 
import json
import os
import datetime
import re
import glob
import csv
import time
import pandas as pd
from nltk.stem import *
from collections import Counter
from functools import reduce
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk import tokenize
import datatable as dt   
import csv


#To read and store dictionary
import pickle

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\auror\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#TODO
#EDA
#do Lemmatization in the Data Cleaning 
#Create all the missing tsv
#Clean and comment the code
#CHECK EFFICIENCY 1.2 and 1.3
#Parse list before saving into

### 1. Data collection
## 1.1. Get the list of places

In [ ]:
total_link = []
s = requests.Session()
for n in tqdm(range(1,401)):
    url = f'https://www.atlasobscura.com/places?page={n}&sort=likes_count'
    result = s.get(url)
    soup = bs(result.text)
    puf = soup.find_all("a", {'class': 'content-card content-card-place'})
    for x in puf:
        total_link.append(x['href'])


In [ ]:
f = open('total_link.txt',"r")
total_link = f.read().split("\n")

## 1.2. Crawl places

In [ ]:
def downloadPage(start,end, array):
    count_link = ((start-1)*18)+1
    count_page = start
    
    parent_dir = f'./all_Pages'
    s = requests.Session()
    header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    for x in tqdm(range(start, end)):
        if count_link%10 == 0:
            s = requests.Session()
        if count_link%18 == 1:
            path = os.path.join(parent_dir, f"folder_{count_page}")
            os.mkdir(path)
        for y in range(18):
            url = f'https://www.atlasobscura.com{array[count_link-1]}'
            name_file = f'location_{count_link}'
            name_folder = f'folder_{count_page}'
            with open(f'./all_Pages/{name_folder}/{name_file}.html', 'w', encoding='utf8') as fp:
                req = s.get(url, headers = header)
                fp.write(req.text)
                if req.status_code != 200:
                     time.sleep(120)
                     req = s.get(url, headers = header)
                fp.write(s.get(url).text)
            if count_link%18 == 0:
                count_page += 1
            count_link += 1
           
        

## 1.3 Parse downloaded pages

In [ ]:

def save_TSV():
    os.mkdir("TSV Files")
    for x in os.listdir("all_Pages"):
        for y in os.listdir(f'all_Pages/{x}'):
            with open(f'./all_Pages/{x}/{y}', encoding='utf8') as f:
                p = f.read()
                soup =  bs(p)
                pageAttribute = []
                placeName = findPlaceName(soup)
                placeTags = findPlaceTags(soup)
                numPeopleVisited = findNumPeopleVisited(soup)
                numPeopleWant = findNumPeopleWant(soup)
                placeDesc = findDescription(soup)
                placeShortDesc = findShortDescription(soup)
                placeNearby = findNearbyPlaces(soup)
                placeAddress = findAddress(soup)
                placeAlt, placeLong = findCordinates(soup)
                placeEditors = findPostEditors(soup)
                placePubDate = findPublishingDate(soup)
                placeRelatedList = findPlaceNear(soup)
                placeRelatedPlaces = findRelatedPlaces(soup)
                placeURL = findPageURL(soup)
                with open(f'./TSV Files/{y[:-5]}.tvs', 'wt', encoding='utf8') as fp:
                    csv.writer(fp, delimiter='\t').writerow([placeName, placeTags, numPeopleVisited, numPeopleWant, placeDesc, placeShortDesc, placeNearby, placeAddress, placeAlt, placeLong, placeEditors, placePubDate, placeRelatedList, placeRelatedPlaces, placeURL])
            

In [ ]:

def download_TSV():
    os.mkdir("TSV Files")
    a = 0
    for path in tqdm(glob.glob(r"all_Pages/*/*")):
        with open(path, encoding='utf8') as f:
                    a += 1
                    p = f.read()
                    soup =  bs(p)
                    pageAttribute = []
                    placeName = findPlaceName(soup)
                    placeTags = findPlaceTags(soup)
                    placeTags = ",".join(placeTags)
                    numPeopleVisited = findNumPeopleVisited(soup)
                    numPeopleWant = findNumPeopleWant(soup)
                    placeDesc = findDescription(soup)
                    placeDesc = " ".join(placeDesc)
                    placeShortDesc = findShortDescription(soup)
                    placeNearby = findNearbyPlaces(soup)
                    placeNearby = ",".join(placeNearby)
                    placeAddress = findAddress(soup)
                    placeAlt, placeLong = findCordinates(soup)
                    placeEditors = findPostEditors(soup)
                    placeEditors = ",".join(placeEditors)
                    placePubDate = findPublishingDate(soup)
                    placeRelatedList = findPlaceNear(soup)
                    placeRelatedList = ",".join(placeRelatedList)
                    placeRelatedPlaces = findRelatedPlaces(soup)
                    placeRelatedPlaces = ",".join(placeRelatedPlaces)
                    placeURL = findPageURL(soup)
                    with open(f'./TSV Files/{a}.tvs', 'wt', encoding='utf8') as fp:
                        csv.writer(fp, delimiter='\t').writerow([placeName, placeTags, numPeopleVisited, numPeopleWant, placeDesc, placeShortDesc, placeNearby, placeAddress, placeAlt, placeLong, placeEditors, placePubDate, placeRelatedList, placeRelatedPlaces, placeURL])
        
        

In [ ]:
download_TSV()

In [ ]:
def findPlaceName(soup):
    placeName = soup.find("h1", {"class": "DDPage__header-title"})
    if placeName != None:
        placeName = placeName.text
    #placeName = re.sub('[A-Za-z0-9_.,! "]*' ,'',placeName)
    return placeName

def findPlaceTags(soup):
    tags = []
    placeTags = soup.find_all("a", {"class": "itemTags__link js-item-tags-link"})
    for tag in placeTags:
        t = tag.text.replace("\n", "")
        #t = re.sub('[A-Za-z0-9 _.,!"]*','',t)
        tags.append(t)
    return tags

def findNumPeopleVisited(soup):
    peopleVisited = soup.find_all("div", {"class": "title-md item-action-count"})
    if len(peopleVisited) > 0:
        peopleVisited = int(peopleVisited[0].text)
    return peopleVisited

def findNumPeopleWant(soup):
    peopleVisited = soup.find_all("div", {"class": "title-md item-action-count"})
    if len(peopleVisited) > 0:
        peopleVisited = int(peopleVisited[1].text)
    return peopleVisited

def findDescription(soup):
    all_description = []
    descriptions = soup.find_all("div", {"class": "DDP__body-copy"})
    for description in descriptions:
        d = description.text.replace("\n","")
        all_description.append(d)
    return all_description

def findShortDescription(soup):
    shortDescription = soup.find("h3", {"class": "DDPage__header-dek"})
    if shortDescription != None:
        shortDescription = shortDescription.text.replace("\n", "")
    return shortDescription

def findNearbyPlaces(soup):
    nearPlaces = []
    nearbyPlaces = soup.find_all("div", {"class": "DDPageSiderailRecirc__item-title"})
    if nearbyPlaces != None:
        for place in nearbyPlaces:
            p = place.text.replace("\n","")
            nearPlaces.append(p)
        #Convert the list to set, and then back again to list to remove repetition
    return set(nearPlaces)

def findAddress(soup):
    strings = []
    adress_strings = soup.find("address", {"class": "DDPageSiderail__address"})
    if adress_strings != None:
        adress_strings = adress_strings.find("div")
        for info in adress_strings:
            s = info.text.replace("\n", "")
            if s != "":
                strings.append(s)
        if len(strings) > 3:
            return " ".join(strings[:3])
    else:
        return " "

def findCordinates(soup):

    cordinates = soup.find("div", {"class":"DDPageSiderail__coordinates js-copy-coordinates"})
    if cordinates != None:
        return cordinates.text.replace("\n", "").replace(" ","").split(",")
    return " ", " "

def findPostEditors(soup):
    all_editors =[]
    editors = soup.find_all("a", {"class":"DDPContributorsList__contributor"})         
    for person in editors:
        s = person.text.replace("\n", "")
        all_editors.append(s)
    return all_editors

def findPublishingDate(soup):
    #Pick the right info
    dateString = soup.find("div", {"class":"DDPContributor__name"}) 
    #Let's clean the string
    if dateString != None:
        s = dateString.text.replace("\n", "")
        #Let's modify it for the right format of datetime
        split = s.split()
        #Let's convert the string Month into the corrispondent number by using "strptime()" 
        split[0] = str(datetime.datetime.strptime(split[0], '%B').month)
        #My format
        format = "%m %d, %Y"
        #Convert from String to datetime
        date = datetime.datetime.strptime(" ".join(split), format)
        return date
    else:
        return ""

def findPlaceNear(soup):
    lists =[]
    relatedLists = soup.find("div", {"data-gtm-template":"DDP Footer Recirc Nearby"})
    if relatedLists != None:
        relatedLists = relatedLists.find_all("h3", {"class":"Card__heading --content-card-v2-title js-title-content"})
        for list in relatedLists:
            s = list.text.replace("\n", "")
            #s = re.sub('[A-Za-z0-9 _.,!"]*','',s)
            lists.append(s)
    return lists

def findRelatedPlaces(soup):
    lists =[]
    relatedLists = soup.find("div", {"data-gtm-template":"DDP Footer Recirc Related"})
    if relatedLists != None:
        relatedLists = relatedLists.find_all("h3", {"class":"Card__heading --content-card-v2-title js-title-content"})
        for list in relatedLists:
            s = list.text.replace("\n", "")
            #s = re.sub('[A-Za-z0-9 _.,!"]*','',s)
            lists.append(s)
    return lists

def findPageURL(soup):
    numVisitedPeople = soup.find("link", {"rel":"canonical"})
    return numVisitedPeople['href']



## Load all the tsv file into one pandas dataframe

We decided to save all the .tsv file as one joined .csv where every file rappresents one row of the .csv file, so that we can work and access the data without having to read 7200 tsv file each time. The data are stored in the "tsv_dataframe"

This mean that the function load_tsv() has only been executed once. The other iteration we just read the .csv file

In [ ]:

def load_tsv():
    tsv = []
    dtypes = {}
    for x in tqdm(os.listdir("TSV Files")):
        df = pd.read_csv(f'TSV Files/{x}',
            sep="\t",
            header=None,
            names=["placeName", "placeTags", "numPeopleVisited", "numPeopleWant", "placeDesc", "placeShortDesc", "placeNearby","placeAdress", "placeAlt", "placeLong", "placeEditors","placePubDate", "placeRelatedList", "placeRelatedPlace", "placeURL"])
        tsv.append(df)

    return pd.concat(tsv)

In [3]:
# data = load_tsv()
data = pd.read_csv("tsv_dataframe.csv",index_col=0)

#Reset Index
data.reset_index(inplace = True, drop=True)

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
data.sort_values('numPeopleWant')

## Data Cleaning

- remove punctuation 
- remove stopwords 
- stemming 

In [5]:


def remove_punctuations(string):
    # first we remove the punctuations
    # in order to do it we need to tekenize the string with the function tokenize and then applying the function RegexpTokenizer
    return RegexpTokenizer(r'\w+').tokenize(string)


def stemming(string):
    # now we move forward with the stemming
    porter = PorterStemmer()
    string_stem=[porter.stem(word) for word in string]
    # we can now return the cleaned string 
    return string_stem

def remove_stopwords(string):
    # after this we can now remove all the stopwords in each word in string_t
    return  [word for word in string if not word.lower() in set(stopwords.words("english"))]
    # now we move forward with the stemming

def cleaning(string):
    #I apply all the function for cleaning the string
    string = string.lower()
    string = remove_punctuations(string)
    string = remove_stopwords(string)
    string = stemming(string)

    #return a list containing all the words of the original string after the cleaning
    return string

## 2.1 

In [6]:
#The first dictionary where every key is a word and it's value is the corrispondant number to that word

def createFirstDic(data):
    dic = {}
    value = 1
    for description in tqdm(data["placeDesc"]):
        #Cleaning the data
        description = cleaning(description)
        for word in description:
            if word in dic.keys():
                continue
            else:
                dic[word] = value
                value += 1
    return dic

In [7]:
dic1 = createFirstDic(data)

100%|██████████████████████████████████████████████████████████████████████████████| 7200/7200 [09:32<00:00, 12.58it/s]


In [ ]:
len(dic1)

## 2.1.1 Creating your index

In [ ]:
def createSecondDic(data, dic1):
    dic2 = {}
    a = 0
    for description in data["placeDesc"]:
        #Cleaning the data
        description= cleaning(description)
        for word in description:
            if dic1[word] in dic2.keys():
                dic2[dic1[word]].add(data.placeName[a])
            else:
                dic2[dic1[word]] = set([data.placeName[a]])
        a += 1
    return dic2

    

In [ ]:
dic2 = createSecondDic(data,dic1)

In [17]:
def save_dic(dic, name):
    with open(f'{name}.pkl', 'wb') as f:
        pickle.dump(dic, f)

In [4]:
def read_dic(name):
    with open(f'{name}.pkl', 'rb') as f:
        loaded_dict = pickle.load(f)
        return loaded_dict

In [19]:
dic1 = read_dic("dic1")

## 2.1 Execute the query

In [5]:
def query(list):
    s = dic2[dic1[list[0]]]
    for x in range(1, len(list)):
        s.intersection(dic2[dic1[list[x]]])
    return s

In [ ]:
s = query(["class", "including"])

res_query = data[data['placeName'].isin(list(s))]

In [ ]:
res_query.head()

## 2.2 Conjunctive query & Ranking score

In [ ]:
def stem_text(txt):
    return [word for word in txt.split(' ')]

In [ ]:
data['list_words'] = data.placeDesc.apply(lambda row: stem_text(row))

In [ ]:

vectorizer = TfidfVectorizer(use_idf=True)
x =vectorizer.fit_transform(data.placeDesc).todense()
df = pd.DataFrame(x, columns = 
vectorizer.get_feature_names())
df
#vectorizer.vocabulary_

In [ ]:
def createInvertedIndex(data, dic1):
    dic2 = {}
    a = 0
    for description in data["placeDesc"]:
        description = description.replace('\'',"")
        description = description.replace('[',"")
        description = description.replace(']',"")
        description = description.lower()
        for word in description.split():
            if dic1[word] in dic2.keys():
                dic2[dic1[word]].add(data.placeName[a])
            else:
                b = []
                dic2[dic1[word]] = set([data.placeName[a]])
            # print(dic2)
        a += 1
    return dic2

## 3. Define a new score!

In [158]:
query_user = list(input("Insert the words that you want to be found in the description's places: ").split())

Insert the words that you want to be found in the description's places: american museum


In [163]:
#first of all we execute the search engine as in 2.1 with the query given by the user
dic1 = read_dic("dic1")
dic2 = read_dic("dic2")
s = query(query_user)
res_query = data[data['placeName'].isin(list(s))]
res_query.reset_index(inplace = True, drop=True) 

In [160]:
n = int(input('What is the top rank visualization that you want to have? \nInsert "1" if you prefer that is by number of tourists. \nInsert "2" if you prefer that is by number of people that want to visit the site.\n'))

What is the top rank visualization that you want to have? 
Insert "1" if you prefer that is by number of tourists. 
Insert "2" if you prefer that is by number of people that want to visit the site.
1


In [161]:
k = int(input('Insert k the number of places that you want to visualize:')) 
m = int(input('Insert "1" if you want to see the k most popular places or insert "2" if you want to see the less famous places:'))

Insert k the number of places that you want to visualize:30
Insert "1" if you want to see the k most popular places or insert "2" if you want to see the less famous places:1


In [151]:
def new_score(res_query,n,k,m):
    scores = []
    if n == 1:
        by = res_query['numPeopleVisited']
    else:
        by = res_query['numPeopleWant']
    max_value = by.max()
    for i in range(res_query.shape[0]):
        scores.append(round(by[i]/max_value,4))
    res_query.insert(res_query.shape[1], "scores", scores)
    if m == 1:
        res_query = res_query.sort_values(by=['scores'],ascending=False)
    else:
        res_query = res_query.sort_values(by=['scores'],ascending=True)
        
    return res_query.head(k)

In [164]:
resquery = new_score(res_query,n,k,m)
resquery.head(k)

,placeName,placeTags,numPeopleVisited,numPeopleWant,placeDesc,placeShortDesc,placeNearby,placeAdress,placeAlt,placeLong,placeEditors,placePubDate,placeRelatedList,placeRelatedPlace,placeURL,scores
673,Alcatraz Island,"al capone,crime museums,lighthouses,prisons,cr...",6639,4018,"Alcatraz, also referred to as “The Rock,” is r...","In all of the 29 years it was in operation, no...","Musée Mécanique,Aquatic Park Tombstones,Magowa...","Alcatraz Island San Francisco, California, 941...",37.8267,-122.4230,"AllisonEng,In Search of the Perfect Sunset,che...",2011-03-06 00:00:00,"Musée Mécanique,Aquatic Park Tombstones,Magowa...","Napier Prison,Captain Joe Byrd Cemetery ,Old P...",https://www.atlasobscura.com/places/alcatraz-i...,1.0000
764,Shakespeare and Company,"shakespeare,bookstores,places to stay,literature",3994,3090,"In 1951, an eccentric American named George Wh...",This iconic Parisian bookstore doubles as the ...,"Rue du Chat-qui-Pêche,Paris Point Zero,The Bou...","37 rue de la Bûcherie Paris, 75005 France",48.8526,2.3471,"Madiha Bataineh,dormeroom,gingercinnamon,comba...",2010-09-24 00:00:00,"Rue du Chat-qui-Pêche,Paris Point Zero,The Bou...","White Rabbit Books & Curiosities,Book and Bed ...",https://www.atlasobscura.com/places/shakespear...,0.6016
84,Garden of the Gods,"roadside attractions,geological oddities,natur...",3890,2232,Colorado is famous for its grand red rock form...,Strange red rock formations that have been att...,"Emma Crawford Coffin Race,Manitou Cliff Dwelli...","1805 N. 30th st. Colorado Springs, Colorado, 8...",38.8784,-104.8698,"smash0gre,nathanherbst,VagabondTabby,JoeBenSim...",2012-08-09 00:00:00,"Emma Crawford Coffin Race,Manitou Cliff Dwelli...","Fantasy Canyon,Chained Rock,The Brutus Stone,W...",https://www.atlasobscura.com/places/garden-of-...,0.5859
729,Multnomah Falls,"natural wonders,water,nature",3558,3331,"Only a 30-minute drive from Portland, Oregon, ...","A roaring, awe-inspiring double cascade of icy...","Chocolate Waterfall,Beacon Rock,Vista House at...","Multnomah Falls, Oregon 97014, USA Corbett, Or...",45.5762,-122.1158,"canuck,Oldbugss,Mark Casey,kayhoskins66,tonyan...",2010-08-22 00:00:00,"Beacon Rock,Vista House at Crown Point,Chocola...","Ayres of Swinister,Cascade Falls,Jusanji Pond,...",https://www.atlasobscura.com/places/multnomah-...,0.5359
793,Blue Mustang,"airports,horses,disasters,sculptures,architect...",3500,1181,The “Blue Mustang” sculpture was created by th...,The Denver airport is guarded by a 32-foot-tal...,"Enchanted Hart Haus ,National Wildlife Propert...","Peña Blvd Denver, Colorado, 80249 United States",39.8342,-104.6759,"ronin,WhiskeyBristles,jenniferliszka,Nicholas ...",2014-11-21 00:00:00,"National Wildlife Property Repository,National...","Music Box Village,Panyer Boy,La Maison d'Adam ...",https://www.atlasobscura.com/places/blue-mustang,0.5272
273,Japanese Tea Garden,"japanese gardens,botanical gardens,horticultur...",3230,2786,The Japanese Tea Garden in Golden Gate Park is...,Relics of the 1894 Midwinter Exposition contin...,"Three Gems,Faery Door,Spreckels Temple of Music","75 Hagiwara Tea Garden Drive San Francisco, Ca...",37.7702,-122.4702,"Claymore,crankyavocado,Rachel,MtnLady,shakespe...",2011-07-26 00:00:00,"Faery Door,Three Gems,Spreckels Temple of Musi...","Mildred E. Mathias Botanical Garden,Bartram's ...",https://www.atlasobscura.com/places/japanese-t...,0.4865
726,Marie Laveau's Tomb,"voodoo,catacombs and crypts,cemeteries",3201,3278,Marie Laveau was a famous and powerful voodoo ...,The final resting place of New Orleans' most f...,"Nicolas Cage's Pyramid Tomb,Storyville,Saint L...",1201-1253 Conti St 425 Basin Street New Orlean...,29.9591,-90.0714,"Eric,Darrell Powers,In Search of the Perfect S...",2016-10-10 00:00:00,"Nicolas Cage's Pyramid Tomb,Saint Louis Cemete...","Nagcarlan Underground Cemetery,Crypt of Dr. Wi...",https://www.atlasobscura.com/places/marie-lave...,0.4822
674,The Witch House of Salem,"witches,witchcraft,memento mori,houses,co